In [1]:
%pip install catboost

import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier, RandomForestClassifier,StackingClassifier,BaggingClassifier,AdaBoostClassifier,ExtraTreesClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, RobustScaler, OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report, make_scorer,mean_squared_error
from catboost import CatBoostClassifier, Pool, cv
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.naive_bayes import GaussianNB, CategoricalNB
from sklearn.metrics import roc_curve, roc_auc_score, auc
# import matplotlib.pyplot as plt
# import lightgbm as lgb
import xgboost as xgb



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:

train_set = pd.read_csv(r'C:\Users\hrahe\Downloads\train_set.csv')
test_set = pd.read_csv(r'C:\Users\hrahe\Downloads\test_set.csv')

In [3]:
categorical_cols = []
numerical_cols = []

for column in train_set.columns:
  if train_set[column].dtype == object or train_set[column].nunique() < 10:
    categorical_cols.append(column)
  else:
    numerical_cols.append(column)

print("Categorical Columns:", categorical_cols)
print("Numerical Columns:", numerical_cols)

Categorical Columns: ['X4', 'X5', 'X6', 'X8', 'X10', 'X11', 'X16', 'Y']
Numerical Columns: ['RecordId', 'X2', 'X3', 'X7', 'X9', 'X12', 'X13', 'X14', 'X15', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32', 'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42', 'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52', 'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62', 'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72', 'X73', 'X74', 'X75', 'X76', 'X77', 'X78']


In [4]:
# Handle missing values for both train and test datasets
for column in categorical_cols:
  if column in train_set.columns and train_set[column].isnull().any():
    mode_imputer = SimpleImputer(strategy='most_frequent')
    train_set[column] = mode_imputer.fit_transform(train_set[[column]])
  if column in test_set.columns and test_set[column].isnull().any():
    if column in train_set.columns:
      mode_imputer = SimpleImputer(strategy='most_frequent')
      test_set[column] = mode_imputer.fit_transform(test_set[[column]])
    else:
      print(f"Warning: Column '{column}' is missing in the training set and cannot be imputed in the test set.")

for column in numerical_cols:
  if column in train_set.columns and train_set[column].isnull().any():
    mean_imputer = SimpleImputer(strategy='mean')
    train_set[column] = mean_imputer.fit_transform(train_set[[column]])
  if column in test_set.columns and test_set[column].isnull().any():
    if column in train_set.columns:
      mean_imputer = SimpleImputer(strategy='mean')
      test_set[column] = mean_imputer.fit_transform(test_set[[column]])
    else:
      print(f"Warning: Column '{column}' is missing in the training set and cannot be imputed in the test set.")


# Find missing values in the training set
missing_values = train_set.isnull().sum()
print(missing_values[missing_values > 0])

# Find missing values in the test set
missing_values = test_set.isnull().sum()
print(missing_values[missing_values > 0])



Series([], dtype: int64)
Series([], dtype: int64)


In [5]:


# Assuming train_set and test_set are pandas DataFrames
# Get all columns except 'Y' for X
X = train_set[[col for col in train_set.columns if col != 'Y']]

# Get only 'Y' column for y
y = train_set['Y']

# Select the same features for the test data
X_testdata = test_set[[col for col in test_set.columns if col != 'Y']]
if 'RecordId' in X.columns:
  X = X.drop('RecordId', axis=1)
if 'RecordId' in X_testdata.columns:
  X_testdata = X_testdata.drop('RecordId', axis=1)

# ... rest of your code (scaling, feature selection, model training, etc.) ...
X.columns

Index(['X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12',
       'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22',
       'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32',
       'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42',
       'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52',
       'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62',
       'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72',
       'X73', 'X74', 'X75', 'X76', 'X77', 'X78'],
      dtype='object')

In [6]:
cols=X.columns
cols

Index(['X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12',
       'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22',
       'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32',
       'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42',
       'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52',
       'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62',
       'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72',
       'X73', 'X74', 'X75', 'X76', 'X77', 'X78'],
      dtype='object')

In [7]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Assuming X and X_testdata are already defined as DataFrames
# Show the columns of X
print("Columns of X before scaling:", X.columns)

# Scale the features using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_testdata_scaled = scaler.transform(X_testdata)

# Convert the scaled arrays back to DataFrames to retain column names
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
X_testdata_scaled_df = pd.DataFrame(X_testdata_scaled, columns=X_testdata.columns)

# Show the columns of the scaled DataFrame
print("Columns of X after scaling:", X_scaled_df.columns)

Columns of X before scaling: Index(['X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12',
       'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22',
       'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32',
       'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42',
       'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52',
       'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62',
       'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72',
       'X73', 'X74', 'X75', 'X76', 'X77', 'X78'],
      dtype='object')
Columns of X after scaling: Index(['X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12',
       'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22',
       'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32',
       'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42',
       'X43', 'X44', 'X45'

In [8]:

X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.2, random_state=2)

In [ ]:
from catboost import CatBoostClassifier

# Initialize the CatBoost classifier with default parameters
catboost_clf = CatBoostClassifier(
    iterations=8000,
    learning_rate=0.035,
    depth=2,
    random_seed=2,
    verbose=100,
   
)

# Fit the model to the training data
catboost_clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = catboost_clf.predict(X_test)
y_pred_proba = catboost_clf.predict_proba(X_test)[:, 1]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f"Accuracy: {accuracy}")
print(f"ROC AUC Score: {roc_auc}")

0:	learn: 0.2054797	total: 198ms	remaining: 26m 25s
100:	learn: 0.0112351	total: 2.32s	remaining: 3m 1s
200:	learn: 0.0100391	total: 4.25s	remaining: 2m 45s
300:	learn: 0.0092536	total: 6.38s	remaining: 2m 43s
400:	learn: 0.0086759	total: 8.53s	remaining: 2m 41s
500:	learn: 0.0081726	total: 10.7s	remaining: 2m 39s
600:	learn: 0.0077233	total: 13s	remaining: 2m 39s
700:	learn: 0.0073306	total: 15.2s	remaining: 2m 38s
800:	learn: 0.0069629	total: 17.5s	remaining: 2m 36s
900:	learn: 0.0066207	total: 19.7s	remaining: 2m 35s
1000:	learn: 0.0063149	total: 23s	remaining: 2m 40s
1100:	learn: 0.0059791	total: 25.3s	remaining: 2m 38s
1200:	learn: 0.0056370	total: 28.1s	remaining: 2m 39s
1300:	learn: 0.0053583	total: 30.9s	remaining: 2m 39s
1400:	learn: 0.0050901	total: 33.2s	remaining: 2m 36s
1500:	learn: 0.0048448	total: 36s	remaining: 2m 35s
1600:	learn: 0.0046042	total: 38.5s	remaining: 2m 33s
1700:	learn: 0.0043804	total: 41.1s	remaining: 2m 32s
1800:	learn: 0.0041562	total: 44s	remaining: 2

In [10]:
from sklearn.feature_selection import SelectFromModel
clf=catboost_clf
# Assuming clf is your trained model
selector = SelectFromModel(clf, max_features=60, prefit=True)
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)
X_testdata_selected = selector.transform(X_testdata)

# Get the selected feature names
selected_features = X_train.columns[selector.get_support()]

print(f"Selected 60 best features:")
print(selected_features)

Selected 60 best features:
Index(['X2', 'X3', 'X7', 'X9', 'X12', 'X13', 'X14', 'X15', 'X21', 'X23', 'X24',
       'X26', 'X28', 'X31', 'X32', 'X37', 'X38', 'X40', 'X42', 'X44', 'X45',
       'X46', 'X47', 'X51', 'X58', 'X60', 'X63', 'X64', 'X65', 'X66', 'X67',
       'X68', 'X69', 'X70', 'X73'],
      dtype='object')


c:\Users\hrahe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
c:\Users\hrahe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
c:\Users\hrahe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [11]:
from sklearn.metrics import roc_auc_score

# Select only the selected features from X_train and X_test
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Train the model with the selected features
clf.fit(X_train_selected, y_train)

# Predict the probabilities for X_test
y_pred_proba = clf.predict_proba(X_test_selected)[:, 1]

# Calculate the AUC value
auc_value = roc_auc_score(y_test, y_pred_proba)

# Print the AUC value
print(f'AUC: {auc_value}')

0:	learn: 0.2055495	total: 18.4ms	remaining: 2m 27s
100:	learn: 0.0114543	total: 1.74s	remaining: 2m 16s
200:	learn: 0.0104737	total: 3.61s	remaining: 2m 20s
300:	learn: 0.0097597	total: 5.37s	remaining: 2m 17s
400:	learn: 0.0092084	total: 7.12s	remaining: 2m 14s
500:	learn: 0.0087375	total: 8.88s	remaining: 2m 12s
600:	learn: 0.0083234	total: 10.6s	remaining: 2m 10s
700:	learn: 0.0079136	total: 12.3s	remaining: 2m 8s
800:	learn: 0.0075276	total: 14.1s	remaining: 2m 6s
900:	learn: 0.0071446	total: 15.8s	remaining: 2m 4s
1000:	learn: 0.0068395	total: 17.6s	remaining: 2m 3s
1100:	learn: 0.0065297	total: 19.3s	remaining: 2m 1s
1200:	learn: 0.0062069	total: 21.2s	remaining: 1m 59s
1300:	learn: 0.0059024	total: 22.9s	remaining: 1m 57s
1400:	learn: 0.0056353	total: 24.6s	remaining: 1m 56s
1500:	learn: 0.0053608	total: 26.6s	remaining: 1m 55s
1600:	learn: 0.0050932	total: 28.9s	remaining: 1m 55s
1700:	learn: 0.0048480	total: 31.2s	remaining: 1m 55s
1800:	learn: 0.0046439	total: 33.5s	remainin

In [12]:
# Select the features from the entire dataset
X_selected = X_scaled_df[selected_features]

# Train the model on the entire dataset with the selected features
clf.fit(X_selected, y)

# Make predictions on the entire dataset
y_pred = clf.predict(X_selected)
y_pred_proba = clf.predict_proba(X_selected)[:, 1]

# Evaluate the model
accuracy = accuracy_score(y, y_pred)
roc_auc = roc_auc_score(y, y_pred_proba)

print(f"Accuracy on entire dataset: {accuracy}")
print(f"ROC AUC Score on entire dataset: {roc_auc}")


0:	learn: 0.2018580	total: 30.3ms	remaining: 4m 2s
100:	learn: 0.0118331	total: 3.58s	remaining: 4m 40s
200:	learn: 0.0108093	total: 6.25s	remaining: 4m 2s
300:	learn: 0.0101697	total: 8.63s	remaining: 3m 40s
400:	learn: 0.0096754	total: 11.5s	remaining: 3m 37s
500:	learn: 0.0092172	total: 14.2s	remaining: 3m 33s
600:	learn: 0.0088345	total: 16.4s	remaining: 3m 22s
700:	learn: 0.0084611	total: 18.5s	remaining: 3m 13s
800:	learn: 0.0081409	total: 20.7s	remaining: 3m 5s
900:	learn: 0.0078504	total: 22.8s	remaining: 2m 59s
1000:	learn: 0.0075824	total: 24.8s	remaining: 2m 53s
1100:	learn: 0.0073076	total: 27.1s	remaining: 2m 49s
1200:	learn: 0.0070488	total: 29.3s	remaining: 2m 46s
1300:	learn: 0.0068061	total: 32s	remaining: 2m 44s
1400:	learn: 0.0065781	total: 34.2s	remaining: 2m 41s
1500:	learn: 0.0063651	total: 36.9s	remaining: 2m 39s
1600:	learn: 0.0061413	total: 39.1s	remaining: 2m 36s
1700:	learn: 0.0059372	total: 41.7s	remaining: 2m 34s
1800:	learn: 0.0057315	total: 43.9s	remainin

In [13]:
# Predict probabilities for the test set
X_testdata_scaled_df_selected=X_testdata_scaled_df[selected_features]
y_pred_proba_testdata = clf.predict_proba(X_testdata_scaled_df_selected)[:, 1]

y_pred_proba_testdata

array([7.49052188e-05, 5.04803945e-06, 1.82857351e-06, ...,
       7.01016651e-06, 3.83121501e-05, 5.95773551e-06])

In [14]:
test_set['Y_probability'] = y_pred_proba_testdata

# Save the DataFrame to a CSV file
test_set[['RecordId', 'Y_probability']].to_csv('test_set_with_probabilities_cat.csv', index=False)

print("Probabilities saved to test_set_with_probabilities.csv")

test_set

Probabilities saved to test_set_with_probabilities.csv


,RecordId,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X70,X71,X72,X73,X74,X75,X76,X77,X78,Y_probability
0,300001,79.000000,17.122318,0,0,1,170.200000,1,700,0,...,0.030000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,7.490522e-05
1,300002,38.000000,43.693579,0,0,1,165.100000,1,814,0,...,0.040000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,5.048039e-06
2,300003,36.064225,23.998944,0,0,1,167.086735,1,662,0,...,0.006948,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,1.828574e-06
3,300004,61.846764,31.693449,0,3,1,182.355708,2,862,0,...,0.033153,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,2.412666e-04
4,300005,71.591991,20.086147,1,0,1,166.704917,2,335,0,...,0.004854,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,2.866371e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105477,405478,35.781006,25.569364,0,0,1,158.542014,1,238,0,...,0.002452,0.0,0.0,0.000000,0.12259,0.0,0.0,0.0,0.0,3.421927e-07
105478,405479,73.000000,30.849795,0,0,0,182.900000,1,114,0,...,0.730000,0.0,0.0,1.000000,0.00000,0.0,0.0,0.0,0.0,1.052665e-01
105479,405480,61.872510,31.609595,0,4,1,155.783272,1,536,0,...,0.079609,0.0,0.0,0.826809,0.00000,0.0,0.0,0.0,0.0,7.010167e-06
105480,405481,58.722824,34.631254,1,0,1,174.474165,2,517,0,...,0.000602,0.0,0.0,0.969882,0.00000,0.0,0.0,0.0,0.0,3.831215e-05


In [15]:
import pandas as pd

# Read the generated CSV file
csv_file_path = 'test_set_with_probabilities_cat.csv'
test_set_with_probabilities = pd.read_csv(csv_file_path)

# Get unique values in the Y_probability column
unique_y_probabilities = test_set_with_probabilities['Y_probability'].unique()

# Print the unique values
print(unique_y_probabilities)
print(test_set_with_probabilities)

[7.49052188e-05 5.04803945e-06 1.82857351e-06 ... 7.01016651e-06
 3.83121501e-05 5.95773551e-06]
        RecordId  Y_probability
0         300001   7.490522e-05
1         300002   5.048039e-06
2         300003   1.828574e-06
3         300004   2.412666e-04
4         300005   2.866371e-06
...          ...            ...
105477    405478   3.421927e-07
105478    405479   1.052665e-01
105479    405480   7.010167e-06
105480    405481   3.831215e-05
105481    405482   5.957736e-06

[105482 rows x 2 columns]
